## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

#### Описание данных
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

### Шаг 1. Откроем файл с данными и изучите общую информацию. 

In [1]:
# считываем файл с данными
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

In [2]:
# посмотрим наглядно на внешницй вид данных
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# выведем информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# посмотрим, есть ли пропуски в ячейках
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

###### Выводы: 
- В столбцах `days_employed` (стаж работы) и `total_income` (месячный доход) равное количество пропущенных значений - **2174**.
- Пропуски могли получиться по вине сотрудника (человеческий фактор), который не внес эти данные, или при выгрузке таблицы, ее склеивании из нескольких сторонних таблиц, когда данные были переданы неверно, а программа в результате обработки заменила некорректные значения на **NaN**.
- Есть предположение, что пропуски находятся в одних и тех же строках. Проверим это в следующем пункте.

In [5]:
# посмотрим на строки с пропущенными значениями
df[df['days_employed'].isnull() & df['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [6]:
# посчитаем количество пропусков по двум столбцам одновременно
df[df['days_employed'].isnull() & df['total_income'].isnull()].shape[0]

2174

###### Выводы: 
- Так и есть, в строках, где пропущен стаж работы `days_employed`, также пропущен и месячный доход `total_income`. Количество строк с такими пропусками равно `2174`.
- Замечаем, что данные в таблице занесены как в верхнем, так и в нижнем регистре. В дальнейшем приведем все строковые значения к нижнему регистру.

In [7]:
# получим основные статистичесие данные по всем столбцам
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


###### Выводы: 
- В столбце `children` есть отрицательное значение детей *-1*. Максимальное значение в столбце равно *20*, что вызывает сомнение в достоверности данных.
- В столбце `days_employed` есть как отрицательные значения, так и очень большие значения. Отрицательные значения возьмем по модулю. Большие значения - это ошибка внесения в базу данных, возможно, как по вине сотрудника банка, так и в связи с ошибками при выгрузке данных (например, из Пенсионного Фонда). Стоит задуматься о качестве вносимых в этот стобец данных.


### Шаг 2. Предобработка данных

Посмотрим, есть ли в таблице дубликаты...

In [8]:
df.duplicated().sum()

54

Да, дубликаты есть. На текущий момент нашлось *54* дубликата.

Рассмотрим каждый столбец в таблице подробнее.

#### Столбец `children` (количество детей в семье)

In [9]:
# посмотрим сколько и каких значений в колонке `children`
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

- Отрицательное значение ребенка *-1* заменим на положительное *+1*.
- Значение 20 детей вызывает сомнение. Изучим столбец подробнее, в частности возраст и пол, чтобы определить насколько реальны эти данные.

In [10]:
df[df['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


Значение столбца `children` равное 20 считаем ошибочным. Например, маловероятно, что девушка в 27 лет имеет 20 детей.

Строки со значением `children` = 20 будут удалены из таблицы.

In [11]:
# удаляем строки со значением детей равное 20
df = df.loc[df['children'] != 20].reset_index(drop=True)

# заменяем отрицательное количество детей -1 на положительное +1
df['children'] = df['children'].replace(-1, 1)

# убеждаемся, что замена прошла успешно, и других аномальных значений не осталось.
df['children'].value_counts()

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

#### Столбец `days_employed` (общий трудовой стаж в днях)
В колонке `days_employed` отрицательные значения заменим на положительные.

In [12]:
# возьмем модуль от значения стажа работы.
df['days_employed'] = df['days_employed'].abs()

Кроме того в колонке `days_employed` есть строки с огромным трудовым стажем, который не может соответствовать реальности.

Также применительно к России работать с трудовой книжкой можно с 16 лет.

In [13]:
# выделим строки, где стаж работы превосходит возраст, с учетом того, что стаж начинается с 16 лет.
df.loc[df['days_employed'] > (df['dob_years'] - 16) * 365]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21429,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21432,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21433,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21442,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


###### Вывод
- Со строками, где стаж не соответствует реальности сделать ничего нельзя. Такие результаты могли появиться с учетом неправильной выгрузки данных, например, изначально время считалось от какой-то универсальной даты (UTC). Ошибка операциониста маловероятна, т.к. вручную внести 6-значные цифры в таком количестве строк достаточно трудно. В Банке стоит разобраться, откуда поступили эти данные, и как там учитывается стаж работы.
- С учётом того, что в проекте не стоит задача выявить влияние стажа работы на возврат кредита, данные оставим без изменений. 
- Помним, что в этом столбце `days_employed` ранее были обнаружены пропуски. К работе над ними приступим позже.

#### Изучим cтолбец `dob_years` (возраст клиента в годах)


In [14]:
# выведем все имеющиеся значения возрастов
df['dob_years'].value_counts()

35    615
40    605
41    605
34    600
38    597
42    594
33    579
39    572
31    558
36    553
44    545
29    543
30    537
48    537
37    533
43    511
50    511
32    508
49    505
28    503
45    494
27    491
52    483
56    482
47    480
54    478
46    472
58    461
57    459
53    458
51    447
55    442
59    442
26    407
60    376
25    356
61    354
62    351
63    269
64    264
24    263
23    253
65    194
66    183
22    183
67    167
21    110
0     100
68     99
69     84
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

- `Из справки Google:` Возрастной ценз в банках Российской Федерации разрешает брать кредит с 18 лет".

In [15]:
# посмотрим сколько в базе заемщиков младше 18 лет. 
df.loc[df['dob_years'] < 18]['dob_years'].value_counts()

0    100
Name: dob_years, dtype: int64

Рассмотрим эти строки

In [16]:
df[df['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1038,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1146,0,934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1172,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1383,0,5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1887,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1895,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


###### Вывод 
- Строки с возрастом `dob_years = *0* ` важны для анализа, т.к. содержат полезную информацию. Эти строки оставляем.


#### Столбцы `education` и `education_id` (уровень образования клиента и идентификатор уровня образования)


In [17]:
# посмотрим какие значения принимает столбец education
df['education'].value_counts()

среднее                13698
высшее                  4706
СРЕДНЕЕ                  770
Среднее                  705
неоконченное высшее      666
ВЫСШЕЕ                   272
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [18]:
# переведем содержимое в нижний регистр
df['education'] = df['education'].str.lower()

In [19]:
# проверяем значения еще раз, убеждаясь в том, что регистр изменился.
df['education'].value_counts()

среднее                15173
высшее                  5246
неоконченное высшее      742
начальное                282
ученая степень             6
Name: education, dtype: int64

In [20]:
# посмотрим какому id какое образование соответствует
df.groupby('education_id')['education'].value_counts()

education_id  education          
0             высшее                  5246
1             среднее                15173
2             неоконченное высшее      742
3             начальное                282
4             ученая степень             6
Name: education, dtype: int64

#### Столбцы `family_status` и  `family_status_id` (семейное положение и идентификатор семейного положения)

In [21]:
# посмотрим какие значения принимает столбец family_status
df['family_status'].value_counts()

женат / замужем          12331
гражданский брак          4165
Не женат / не замужем     2804
в разводе                 1193
вдовец / вдова             956
Name: family_status, dtype: int64

In [22]:
# переведем содержимое в нижний регистр
df['family_status'] = df['family_status'].str.lower()

In [23]:
# посмотрим какому id какой семейный статус соответствует
df.groupby('family_status_id')['family_status'].value_counts()

family_status_id  family_status        
0                 женат / замужем          12331
1                 гражданский брак          4165
2                 вдовец / вдова             956
3                 в разводе                 1193
4                 не женат / не замужем     2804
Name: family_status, dtype: int64

#### Столбец `gender` (пол клиента)

In [24]:
# посмотрим какие значения принимает столбец gender
df['gender'].value_counts()

F      14189
M       7259
XNA        1
Name: gender, dtype: int64

In [25]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10672,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Есть странная запись с полом `gender` = 'XNA'. Запись оставим как есть, т.к. пол нас не интересует при текущем анализе данных по возврату кредита, но обратим на этом внимание стоит, чтобы в будущем количество ячеек с таким значением не росло.

### Столбец `income_type` (тип занятости)

In [26]:
# посмотрим какие значения принимает столбец income_type
df['income_type'].value_counts()

сотрудник          11076
компаньон           5063
пенсионер           3847
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Видим, что есть малая выборка среди предпринимателей, безработных, в декрете и студентов. Посмотрим информацию о них.

In [27]:
# выведем строки с income_type = безработный, в декрете, предприниматель, студент
df.loc[df['income_type'].isin(['безработный','в декрете','предприниматель','студент'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3128,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
5921,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
9384,0,578.751554,22,высшее,0,не женат / не замужем,4,M,студент,0,98201.625314,строительство собственной недвижимости
14748,0,395302.838654,45,высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью
18633,0,520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы
20774,2,3296.759962,39,среднее,1,женат / замужем,0,F,в декрете,1,53829.130729,автомобиль


Для чистоты исследования удалим этим строки, т.к. выборка по таким `income_type` очень мала.

In [28]:
df = df.loc[df['income_type'].isin(['безработный','в декрете','предприниматель','студент']) == False]

In [29]:
# посмотрим какие значения после удаления строк принимает столбец income_type
df['income_type'].value_counts()

сотрудник      11076
компаньон       5063
пенсионер       3847
госслужащий     1457
Name: income_type, dtype: int64

#### Столбец `debt` (задолженность по возврату кредитов)

In [30]:
# посмотрим какие значения принимает столбец debt
df['debt'].value_counts()

0    19712
1     1731
Name: debt, dtype: int64

Здесь данные чистые:
- *0* - нет задолженности.
- *1* - задолженность есть. 

#### Столбец `total_income` ( ежемесячный доход)


In [31]:
# проверим, присутствуют ли отрицательные значения total_income < 0 
df.loc[df['total_income'] <= 0] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Нет, таких значений нет. Помним, что в данном столбце есть пропуски значений. К работе над ними приступим позже.

#### Столбец `purpose` (цель получения кредита)

In [32]:
# переведем содержимое в нижний регистр
df['purpose'] = df['purpose'].str.lower()

# посмотрим какие значения принимает столбец debt
df['purpose'].value_counts()

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           770
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         649
операции с коммерческой недвижимостью     647
покупка жилья                             644
жилье                                     643
покупка жилья для семьи                   640
недвижимость                              633
операции со своей недвижимостью           629
строительство собственной недвижимости    628
строительство жилой недвижимости          625
покупка недвижимости                      622
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              609
покупка жилой недвижимости                604
на покупку своего автомобиля              505
заняться высшим образованием      

Основные цели - это недвижимость, автомобиль, свадьба, образование.

### Обработка пропусков

In [33]:
# посчитаем количество пропусков на текущем этапе
df.isnull().sum()

children               0
days_employed       2164
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2164
purpose                0
dtype: int64

Посмотрим какой `income_type` *(тип занятости)* в строках с пропусками.

In [34]:
df.loc[df['days_employed'].isnull()].groupby('income_type')['income_type'].count()

income_type
госслужащий     146
компаньон       505
пенсионер       411
сотрудник      1102
Name: income_type, dtype: int64

- Пропуски есть во всех категориях "трудящихся".

- Разумно будет заменить пропуски по зарплате `total_income` и по трудовому стажу `days_employed` медианным значением по каждому типу занятости `income_type`.

- Хотелось бы соотнести трудовой стаж с возрастом заемщика, однако в столбце трудового стажа `days_employed` у нас есть как нулевые значения, так и очень большие выбросы.

In [35]:
# напишем функцию, которая на вход получает тип занятости `income_type`, и выполняет замену пропусков в этой категории трудящихся.
def pass_replace(income_type):
    # рассчитаем медианные значения для стажа и уровня дохода,
    df_income_type = df.loc[(df['income_type'] == income_type) & (df['income_type'].notnull())]
    median_total_income = df_income_type['total_income'].median()
    median_days_employed = df_income_type['days_employed'].median()
    # заменяем пропуски на соответствующие медианные переменные 
    df['total_income'].loc[df['income_type'] == income_type] = df['total_income'].loc[df['income_type'] == income_type].fillna(median_total_income)
    df['days_employed'].loc[df['income_type'] == income_type] = df['days_employed'].loc[df['income_type'] == income_type].fillna(median_days_employed)
    return print('Пропуски заменены')

In [36]:
#применим функцию к каждому типу занятости
pass_replace('госслужащий')
pass_replace('компаньон')
pass_replace('пенсионер')
pass_replace('сотрудник')

Пропуски заменены
Пропуски заменены
Пропуски заменены
Пропуски заменены


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [37]:
# проверим еще раз наличие пропусков
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски успешно заполнены.

### Обработка дубликатов

In [38]:
# посмотрим, есть ли в таблице дубликаты
df.duplicated().sum()

71

In [39]:
# удалим дубликаты
df = df.drop_duplicates().reset_index(drop = True)
df.shape

(21372, 12)

Дубликатов после подготовки данных нашлось больше (`71` вместо ранее найденных `54`). 

### Замена типа данных

In [40]:
# выведем типы данных у столбцов таблицы
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

Столбцы `days_employed` и `total_income` имеют вещественный тип данных. Заменим его на целочисленный с помощью метода *.astype()*. Ошибки при такой замене возникнуть не должно.

In [41]:
# меняем тип данных столбца total_income и days_employed
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')

#### Поместим очищенный датасет в переменную `cleaned_dataset`.

In [42]:
cleaned_dataset = df

### Лемматизация

In [43]:
# импортируем библиотеку pymystem3
from pymystem3 import Mystem
m = Mystem()

In [44]:
# посмотрим какие значения принимает столбец 'purpose'
cleaned_dataset['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           761
операции с недвижимостью                  674
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         648
операции с коммерческой недвижимостью     646
покупка жилья                             643
жилье                                     642
покупка жилья для семьи                   637
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          623
покупка своего жилья                      620
покупка недвижимости                      619
строительство недвижимости                619
ремонт жилью                              604
покупка жилой недвижимости                603
на покупку своего автомобиля              505
заняться высшим образованием      

Визуально выделим основные непересекающиеся цели, на которые заемщики брали кредит. Поместим значение этих целей в массив `words_lemma`.

In [45]:
words_lemma = ['свадьба', 'жилье', 'автомобиль', 'образование', 'недвижимость']

In [46]:
# напишем функцию, которая принимает на вход текст и выводит одно ключевое слово.
 
def lemmatize(text):
    lemma = m.lemmatize(text) # лемматизируем входящий текст
    for word in words_lemma: # перебираем каждую цель кредита из массива `words_lemma` 
        if word in lemma:  
            lemma = word # в случае совпадения цели кредита и леммы запоминаем новое значение `lemma`
    return lemma

In [47]:
# лемматизируем столбец 'purpose_group'
# выведем в нем основную цель кредита с помощью функции метода `.apply()` и ранее созданной функции `lemmatize`
cleaned_dataset['purpose_group'] = cleaned_dataset['purpose'].apply(lemmatize)
cleaned_dataset.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [48]:
# выведем какие значение и какое количество раз принимает столбец 'purpose_group'
cleaned_dataset['purpose_group'].value_counts()

недвижимость    6328
жилье           4443
автомобиль      4289
образование     3998
свадьба         2314
Name: purpose_group, dtype: int64

In [49]:
# переименуем цель кредита `жилье` в `недвижимость`.
cleaned_dataset['purpose_group'] = cleaned_dataset['purpose_group'].replace('жилье', 'недвижимость')

In [50]:
# еще раз выведем значения столбца 'purpose_group'
cleaned_dataset['purpose_group'].value_counts()

недвижимость    10771
автомобиль       4289
образование      3998
свадьба          2314
Name: purpose_group, dtype: int64

В результате получили категоризацию заемщиков по цели кредита.

## Категоризация данных

В соответствии с тех. заданием сделаем категоризацию данных по количеству детей, семейному положению и уровню дохода.
Категоризацию по цели кредита сделали на предыдущем шаге.

#### Категоризация заемщиков по количеству детей

Выделим 4 категории заемщиков:
- без детей
- с 1 ребенком
- с 2 детьми
- многодетный

In [51]:
# напишем функцию `children_group`, которая на вход принимает количество детей `children` и возвращает категорию заёмщика.
def children_group(children):
    if children == 0:
        return 'без детей'
    if (children == 1):
        return 'с 1 ребенком'
    if (children == 2):
        return 'с 2 детьми'
    return 'многодетный'

In [52]:
# проверим работу функции
print(children_group(3))
print(children_group(2))
print(children_group(1))
print(children_group(0))

многодетный
с 2 детьми
с 1 ребенком
без детей


In [53]:
# применим созданную функцию `children_group` на столбец 'children_group'
cleaned_dataset['children_group'] = cleaned_dataset['children'].apply(children_group)

In [54]:
# посмотрим, что получилось. Выведем первые три строки таблицы.
cleaned_dataset.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,с 1 ребенком
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,с 1 ребенком
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,без детей


In [55]:
# выведем количество всех значений столбца 'children_group'
cleaned_dataset['children_group'].value_counts()

без детей       14087
с 1 ребенком     4854
с 2 детьми       2051
многодетный       380
Name: children_group, dtype: int64

#### Категоризация по семейному положению

In [56]:
# еще раз посмотрим, какие статусы у заемщиков бывают
cleaned_dataset['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Напишем функцию `family_status_group`, которая на вход принимает статус заемщика `family_status` и возвращает категорию заёмщика.
- Статусы `вдовец / вдова`, `в разводе` и `не женат / не замужем` объединим в один с названием `не в браке`. Так как в момент выдачи кредита бывшие мужья или жены не являются поручителями и не могут разделить ответственность за возврат кредита, то такое объединение статусов считаю разумным.
- Статус `гражданский брак` никак не закреплен в семейном кодексе РФ, и подтвердить его невозможно. Поэтому также изменим его на статус `не в браке`.

В итоге у нас будет только два значения: `состоит в браке` и `не состоит в браке`. 

In [57]:
# напишем функцию `family_status_group`, которая на вход принимает статус заемщика `family_status` и возвращает категорию заёмщика.
def family_status_group(family_status):
    if family_status == 'женат / замужем':
        return 'состоит в браке'
    return 'не состоит в браке'

In [58]:
# проверим работу функции
print(family_status_group('женат / замужем'))
print(family_status_group('в разводе'))

состоит в браке
не состоит в браке


In [59]:
# применим созданную функцию `family_status_group` на столбец 'family_status'
cleaned_dataset['family_status_group'] = cleaned_dataset['family_status'].apply(family_status_group)

In [60]:
# посмотрим, что получилось.
cleaned_dataset.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_group,family_status_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,с 1 ребенком,состоит в браке
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,с 1 ребенком,состоит в браке
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,без детей,состоит в браке
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетный,состоит в браке
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,без детей,не состоит в браке


In [61]:
# выведем количество всех значений столбца 'family_status_group'
cleaned_dataset['family_status_group'].value_counts()

состоит в браке       12287
не состоит в браке     9085
Name: family_status_group, dtype: int64

#### Категоризация по месячному доходу

In [62]:
# выведем статистические данные заемщиков по доходу
cleaned_dataset['total_income'].describe()

count    2.137200e+04
mean     1.652965e+05
std      9.821216e+04
min      2.066700e+04
25%      1.075595e+05
50%      1.425940e+05
75%      1.957835e+05
max      2.265604e+06
Name: total_income, dtype: float64

Условимся разделить заемщиков на следующие категории:
- ниже 25% - малый доход,
- от 25% до 40% - ниже среднего,
- от 40% до 60% - средний,
- от 60% до 75% - выше среднего,
- от 75% и выше - высокий доход,

где 50% - это медианное значение, а 25% и 75% - соответствующие квартили.

In [63]:
# создадим переменные в соответствии с выделенными категориями заемщиков. Это позволит сократить время на расчеты. 
quantile_25 = cleaned_dataset['total_income'].quantile([.25]).sum()
quantile_40 = cleaned_dataset['total_income'].quantile([.40]).sum()
quantile_60 = cleaned_dataset['total_income'].quantile([.60]).sum()
quantile_75 = cleaned_dataset['total_income'].quantile([.75]).sum()

In [64]:
# напишем функцию `total_income_group`, которая на вход принимает доход заемщика `total_income` и возвращает категорию заёмщика.
def total_income_group(total_income):
    if total_income <= quantile_25:
        return 'малый достаток'
    if total_income <= quantile_40:
        return 'ниже среднего'
    if total_income <= quantile_60:
        return 'cредний'
    if total_income <= quantile_75:
        return 'выше среднего'
    return 'высокий доход'

In [65]:
# проверим работу функции
print(total_income_group(2.066700e+04))
print(total_income_group(1.075170e+05))
print(total_income_group(1.425870e+05))
print(total_income_group(2.265604e+06))

малый достаток
малый достаток
cредний
высокий доход


In [66]:
# применим созданную функцию `family_status_group` на столбец 'family_status'
cleaned_dataset['total_income_group'] = cleaned_dataset['total_income'].apply(total_income_group)

In [67]:
# посмотрим, что получилось.
cleaned_dataset.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_group,family_status_group,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,с 1 ребенком,состоит в браке,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,с 1 ребенком,состоит в браке,ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,без детей,состоит в браке,cредний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетный,состоит в браке,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,без детей,не состоит в браке,cредний


In [68]:
# выведем количество всех значений столбца 'total_income_group'
cleaned_dataset['total_income_group'].value_counts()

малый достаток    5343
высокий доход     5343
cредний           4274
выше среднего     3206
ниже среднего     3206
Name: total_income_group, dtype: int64

### Шаг 3. Ответьте на вопросы

###### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [69]:
#Построим сводную таблицу зависимости задолженности от количества детей
cleaned_dataset_pivot = cleaned_dataset.pivot_table(index = ['children_group'], columns = 'debt', values = 'family_status_id', aggfunc = 'count')
cleaned_dataset_pivot

debt,0,1
children_group,,
без детей,13024,1063
многодетный,349,31
с 1 ребенком,4410,444
с 2 детьми,1858,193


In [70]:
#добавим дополнительный столбец 'ratio'- отношение должников к сумме всех заемщиков
cleaned_dataset_pivot['ratio'] = (cleaned_dataset_pivot[1] / (cleaned_dataset_pivot[0] + cleaned_dataset_pivot[1]))
cleaned_dataset_pivot.sort_values('ratio', ascending = True)

debt,0,1,ratio
children_group,,,
без детей,13024,1063,0.075460
многодетный,349,31,0.081579
с 1 ребенком,4410,444,0.091471
с 2 детьми,1858,193,0.094100


### Вывод

- Лучше всех кредиты возвращают заемщики без детей. Нет детей - нет непредвиденных расходов.
- Следующее место по отсутствию задолженности занимают многодетные семьи. Возможно это связано с дотациями государства, а также более серьезному подходу заемщиков к взятию кредита и его возврату.
- Заемщики с 1 ребенком и 2 детьми имеют схожие показатели и выступают должниками чаще других. Почему так, однозначно сложно сказать, надо делать перекрестный анализ по другим столбцам. Как вариант, менее серьезный подход к кредиту.

###### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [71]:
# построим сводную таблицу зависимости задолженности от семейного положения
cleaned_dataset_pivot = cleaned_dataset.pivot_table(index = ['family_status_group'], columns = 'debt', values = 'family_status_id', aggfunc = 'count')
cleaned_dataset_pivot

debt,0,1
family_status_group,,
не состоит в браке,8280,805
состоит в браке,11361,926


In [72]:
#добавим дополнительный столбец 'ratio'- отношение должников к сумме всех заемщиков
cleaned_dataset_pivot['ratio'] = (cleaned_dataset_pivot[1] / (cleaned_dataset_pivot[0] + cleaned_dataset_pivot[1]))
cleaned_dataset_pivot.sort_values('ratio', ascending = True)

debt,0,1,ratio
family_status_group,,,
состоит в браке,11361,926,0.075364
не состоит в браке,8280,805,0.088608


### Вывод

Заёмщики состоящие в браке возвращают кредит чаще. Это можно объяснить тем, что в браке кредит вернуть легче, т.к. доходы двоих людей в сумме больше. Кроме того, боязнь понести ответственность за невозврат кредита присутствует у каждого из супругов.

###### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [73]:
# построим сводную таблицу зависимости задолженности от дохода
cleaned_dataset_pivot = cleaned_dataset.pivot_table(index = ['total_income_group'], columns = 'debt', values = 'family_status_id', aggfunc = 'count')
cleaned_dataset_pivot

debt,0,1
total_income_group,,
cредний,3900,374
высокий доход,4962,381
выше среднего,2930,276
малый достаток,4918,425
ниже среднего,2931,275


In [74]:
cleaned_dataset_pivot['ratio'] = (cleaned_dataset_pivot[1] / (cleaned_dataset_pivot[0] + cleaned_dataset_pivot[1]))
cleaned_dataset_pivot.sort_values('ratio', ascending = True)

debt,0,1,ratio
total_income_group,,,
высокий доход,4962,381,0.071308
малый достаток,4918,425,0.079543
ниже среднего,2931,275,0.085777
выше среднего,2930,276,0.086089
cредний,3900,374,0.087506


### Вывод

- Заемщики с высоким доходом возвращают кредит чаще. Думаю, это связано с тем, что это им проще сделать, т.к. доход большой. Тут интереснее было бы соотнести уровень дохода и размер кредита, такая информация дала бы объективнее результаты.
- Заемщики с малым достатком занимают второе место по возврату кредита. Возможно, что такие заемщики берут малые суммы, которые точно смогут вернуть. Не имея высокого дохода, заемщик много раз подумает, прежде, чем брать кредит. Возможны дотации от государства в этой категории граждан.
- При приближении к медианному значению дохода возвращаемость кредитов падает. Объяснить можно тем, что заемщики со средним уровнем дохода плохо оценивают свои шансы на возврат кредита. Представляя средний рабочий класс в обществе, больше других зависимы от государственных реформ, налогов, трудовой миграции и т.д. 

###### Как разные цели кредита влияют на его возврат в срок?

In [75]:
# построим сводную таблицу зависимости задолженности от цели кредита
cleaned_dataset_pivot = cleaned_dataset.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'family_status_id', aggfunc = 'count')
cleaned_dataset_pivot

debt,0,1
purpose_group,,
автомобиль,3889,400
недвижимость,9992,779
образование,3629,369
свадьба,2131,183


In [76]:
#добавим дополнительный столбец 'ratio'- отношение должников к сумме всех заемщиков
cleaned_dataset_pivot['ratio'] = (cleaned_dataset_pivot[1] / (cleaned_dataset_pivot[0] + cleaned_dataset_pivot[1]))
cleaned_dataset_pivot.sort_values('ratio', ascending = True)

debt,0,1,ratio
purpose_group,,,
недвижимость,9992,779,0.072324
свадьба,2131,183,0.079084
образование,3629,369,0.092296
автомобиль,3889,400,0.093262


### Вывод

- Чаще всего возвращают кредит, взятый на недвижимость. Это связано с тем, что такой кредит более других забюрократизирован. Например, ипотека. Дотации государства и четкий расчитанный график платежей в соответствии со всеми параметрами заемщика приносит положительный результат.
- Возврат кредита на свадьбу занимает у нас второе место. Думаю, это связано с тем, что большая часть кредита гасится сразу после свадьбы. Можно сказать "Спасибо!" гостям за денежные подарки молодоженам. Хорошо было бы изучить статистику погашений по дням.
- Кредит на образование и на автомобиль занимают последние места по возврату.
  - С автомобилем, думаю, банк идет навстречу заемщику, закрывая глаза на какие-то характеристики. Автомобиль закаскованный, и в случае невозврата кредита будет изъят и продан на рынке. Ликвидность автомобиля выше, чем недвижимости.
  - С образованием не так однозначно. Я не знаю, как регулируется кредит на образование, и какие документы необходимо предоставить. Возможно процент на такой кредит ниже, чем на потребительский, чем пользуются непорядочные заемщики. Таким образом, на более рискованные операции выдаются кредиты с низким процентом. Надо отдельно разбираться с этой категорией кредитов и более детально.

### Шаг 4. Общий вывод

- Открыт и изучен файл с данными.

В таблице присутствовали пропуски, дубликаты и ошибочные значения, которые были устранены. Стоит обратить внимание на качество исходной базы данных, т.к. ошибки в столбцах `ежемесячный доход`, `возраст заемщика` и `трудовой стаж` недопустимы. Для операционистов, которые оформляют кредит, организовать работу так, чтобы они выбирали параметры из выпадающего списка. Если часть данных подгружается из других источников, то стоит обратить внимание на программную часть, и согласовать условия взаимодействия с другими сервисами или отделами. В таблице также нет данных по сумме кредита, выданного каждого заемщику. Поэтому анализ данных выглядит несколько однобоким.

- Определены основные цели, на которые выдается кредит: `недвижимость, автомобиль, свадьба и образование.`

- Выявлены зависимости возврата кредита от наличия детей у заемщика, семейного положения, уровня дохода и цели кредита.

Чаще других кредит возвращают, если:

- заемщик вообще не имеет детей,
- состоит в браке,
- кредит взят на операции с недвижимостью,
- уровень дохода у заемщика высокий.

Стоит обратить внимание, что "средний класс" заемщиков, как по уровню дохода, так и по количеству детей несет наибольшие риски невозврата.